In [2]:
import pandas as pd
import folium as fm
import re

In [3]:
import os
os.path.exists('./Data/P3_GrantExport.csv')
data = pd.read_csv('./Data/P3_GrantExport.csv', delimiter=';', na_values=['.'], error_bad_lines=False)

In [13]:
universities = data['University'].unique()
len(universities)

78

In [ ]:
URL = "api.geonames.org/postalCodeSearch?"
feature_code = "featureCode=UNIV"
country = "country=CH"


In [ ]:
topo_path = r"Data/ch-cantons.topojson.json"

In [64]:
universities = data['University'].tolist()[:-1]
universities[-1]

ids = set()
for record in universities:
    ids.add(str(record).split()[-1])

ids

 

{'AGS',
 'AORI',
 'ASP',
 'ASPIT',
 'BE',
 'BFH',
 'BITG',
 'BS',
 'CREALP',
 'CSEM',
 'EAWAG',
 'EHB',
 'EMPA',
 'EOC',
 'EPFL',
 'ETHZ',
 'FFHS',
 'FHKD',
 'FHNW',
 'FHO',
 'FIBL',
 'FINST',
 'FMI',
 'FORS',
 'FP',
 'FR',
 'FTL',
 'FUS',
 'GE',
 'HEPBEJUNE',
 'HEPFR',
 'HEPL',
 'HES-SO',
 'HSLU',
 'HfH',
 'IDIAP',
 'IHEID',
 'IKG',
 'IRO',
 'IRSOL',
 'ISR',
 'ISSKA',
 'IST',
 'IUKB',
 'KSPSG',
 'LA',
 'LU',
 'NA',
 'NE',
 'NPO',
 'PHBern',
 'PHFHNW',
 'PHGR',
 'PHLU',
 'PHSG',
 'PHSH',
 'PHSZ',
 'PHTG',
 'PHVS',
 'PHZFH',
 'PHZG',
 'PMOD',
 'PSI',
 'RWS',
 'SAGW',
 'SG',
 'SHLR',
 'SIAF',
 'SIB',
 'SIK-ISEA',
 'SPF',
 'STHB',
 'SUPSI',
 'USI',
 'WSL',
 'ZFH',
 'ZH',
 'nan'}

In [37]:
ids

<generator object <genexpr> at 0x000001D828C7F048>